In [2]:
pip install datasets

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.6.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached multidict-6.4.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached yarl-1.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (72 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta

In [5]:
from datasets import load_dataset

# Load the full dataset
dataset = load_dataset("jpawan33/fkr30k-image-captioning-dataset", split="train")

# Select only the first 5k examples
dataset_small = dataset.select(range(1000))

print(len(dataset_small))  # Should print 5000
dataset_small.save_to_disk("fkr5k-dataset")


1000


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 10656.31 examples/s]


In [2]:
pip install googletrans==4.0.0-rc1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17452 sha256=74479fb0e2cafad68a03bee11726c1888419a8475c0df00c2c08a4177bc5a672
  Stored in directory: /home/vitoupro/.cache/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_from_disk
import os
import json
from PIL import Image
from tqdm import tqdm
import time

# Use Google Translate
from googletrans import Translator

# Load dataset
dataset = load_from_disk("fkr5k-dataset")

# Prepare output folders
os.makedirs("dataset_images", exist_ok=True)

# Translator setup
translator = Translator()

# Progress files
progress_file = "progress.json"
output_txt_file = "translated_dataset.txt"

# Load progress if exists
translated_lines = []
start_idx = 0

if os.path.exists(progress_file):
    with open(progress_file, "r", encoding="utf-8") as f:
        progress = json.load(f)
        translated_lines = progress.get("translated_lines", [])
        start_idx = len(translated_lines)
    print(f"Resuming translation from index {start_idx}...")
else:
    print("Starting new translation...")

# Start translating
for idx, example in enumerate(tqdm(dataset, desc="Translating dataset")):
    if idx < start_idx:
        continue  # Already translated

    text = example["text"]
    image = example["image"]

    try:
        translated_text = translator.translate(text, src='en', dest='km').text
    except Exception as e:
        print(f"Error translating '{text}': {e}")
        translated_text = None

    if translated_text is not None:
        # Save image
        image_path = f"dataset_images/image_{idx+1}.jpg"
        image.save(image_path)

        # Save translation
        line = f"image_{idx+1}.jpg \"{translated_text}\""
        translated_lines.append(line)

        # Save progress every 50 items
        if idx % 50 == 0:
            with open(progress_file, "w", encoding="utf-8") as f:
                json.dump({"translated_lines": translated_lines}, f, ensure_ascii=False, indent=2)

    else:
        print(f"Skipping index {idx} due to translation error.")

    # Optional: delay
    time.sleep(0.5)

# Save final text file
try:
    with open(output_txt_file, "w", encoding="utf-8") as f:
        for line in translated_lines:
            f.write(line + "\n")
    print(f"✅ All translations saved successfully to {output_txt_file}")
except Exception as e:
    print(f"Error saving final text file: {e}")

# Clean up progress file
if os.path.exists(progress_file):
    os.remove(progress_file)


Resuming translation from index 51...


Translating dataset:  37%|███▋      | 373/1000 [09:55<20:04,  1.92s/it]

Error translating 'Men with reflective safety jackets on are working on a street intersection with many orange reflective cones .': The read operation timed out
Skipping index 373 due to translation error.


Translating dataset:  55%|█████▍    | 549/1000 [15:33<14:51,  1.98s/it]

Error translating 'A man leans against a rock pillar with his back to an ongoing parade decorated with white and pink balloon arch .': the JSON object must be str, bytes or bytearray, not NoneType
Skipping index 549 due to translation error.


Translating dataset: 100%|██████████| 1000/1000 [30:02<00:00,  1.80s/it]

✅ All translations saved successfully to translated_dataset.txt


In [2]:
import re
import os

# File to clean
input_file = "translated_dataset.txt"

with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Function to clean text and fix filenames
def clean_line(line):
    # First, split between image name and caption
    parts = line.strip().split(" ", 1)
    
    if len(parts) != 2:
        return None  # Skip malformed lines

    image_name, caption = parts

    # Fix the image name: insert underscore and .jpg properly
    # Assuming image name format is like 'image1jpg' -> 'image_1.jpg'
    if image_name.startswith("image") and image_name.endswith("jpg"):
        index = image_name[5:-3]  # Extract number between 'image' and 'jpg'
        fixed_image_name = f"image_{index}.jpg"
    else:
        fixed_image_name = image_name  # fallback if strange format

    # Clean the caption: remove " and special characters
    caption = caption.replace('"', '')
    caption = re.sub(r"[!@#$%^&*()_+=\[\]{}\\|:;'<>,.?/~`]", '', caption)
    caption = caption.strip()

    return f"{fixed_image_name} {caption}"

# Clean all lines
cleaned_lines = []
for line in lines:
    cleaned = clean_line(line)
    if cleaned:
        cleaned_lines.append(cleaned)

# Overwrite the original file
with open(input_file, "w", encoding="utf-8") as f:
    for line in cleaned_lines:
        f.write(line + "\n")

print(f"✅ Successfully cleaned and fixed filenames in {input_file}")


✅ Successfully cleaned and fixed filenames in translated_dataset.txt


In [1]:
from datasets import load_dataset

# Load the full dataset
dataset = load_dataset("jpawan33/fkr30k-image-captioning-dataset", split="train")

# Select only the first 5k examples
dataset_small = dataset.select(range(5000))

print(len(dataset_small))  # Should print 5000
dataset_small.save_to_disk("fkr5k-dataset-5k")

/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


5000


Saving the dataset (1/1 shards): 100%|██████████| 5000/5000 [00:00<00:00, 11071.60 examples/s]


In [1]:
from datasets import load_from_disk
import os
import json
from PIL import Image
from tqdm import tqdm
import time

# Use Google Translate
from googletrans import Translator

# Load dataset
dataset = load_from_disk("fkr5k-dataset-5k")

# Prepare output folders
os.makedirs("dataset_images", exist_ok=True)

# Translator setup
translator = Translator()

# Progress files
progress_file = "progress5k.json"
output_txt_file = "translated_dataset5k.txt"

# Load progress if exists
translated_lines = []
start_idx = 0

if os.path.exists(progress_file):
    with open(progress_file, "r", encoding="utf-8") as f:
        progress = json.load(f)
        translated_lines = progress.get("translated_lines", [])
        start_idx = len(translated_lines)
    print(f"Resuming translation from index {start_idx}...")
else:
    print("Starting new translation...")

# Start translating
for idx, example in enumerate(tqdm(dataset, desc="Translating dataset")):
    if idx < start_idx:
        continue  # Already translated

    text = example["text"]
    image = example["image"]

    try:
        translated_text = translator.translate(text, src='en', dest='km').text
    except Exception as e:
        print(f"Error translating '{text}': {e}")
        translated_text = None

    if translated_text is not None:
        # Save image
        image_path = f"dataset_images/image_{idx+1}.jpg"
        image.save(image_path)

        # Save translation
        line = f"image_{idx+1}.jpg \"{translated_text}\""
        translated_lines.append(line)

        # Save progress every 50 items
        if idx % 50 == 0:
            with open(progress_file, "w", encoding="utf-8") as f:
                json.dump({"translated_lines": translated_lines}, f, ensure_ascii=False, indent=2)

    else:
        print(f"Skipping index {idx} due to translation error.")

    # Optional: delay
    time.sleep(0.5)

# Save final text file
try:
    with open(output_txt_file, "w", encoding="utf-8") as f:
        for line in translated_lines:
            f.write(line + "\n")
    print(f"✅ All translations saved successfully to {output_txt_file}")
except Exception as e:
    print(f"Error saving final text file: {e}")

# Clean up progress file
if os.path.exists(progress_file):
    os.remove(progress_file)


/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Resuming translation from index 51...


Translating dataset:  23%|██▎       | 1151/5000 [25:57<2:12:15,  2.06s/it]

Error translating 'A small boy wearing a diaper stands near the door and is covered in marker .': The read operation timed out
Skipping index 1151 due to translation error.


Translating dataset:  50%|████▉     | 2485/5000 [1:08:20<1:03:54,  1.52s/it]

...

Error translating 'A female police officer , wearing an officer 's hat and sunglasses , stands in uniform in front of a window-lined street block .': the JSON object must be str, bytes or bytearray, not NoneType
Skipping index 4477 due to translation error.


Translating dataset:  92%|█████████▏| 4584/5000 [2:17:09<10:46,  1.55s/it]

Error translating 'An Asian store with people walking throughout and a big red arrow pointing left .': The read operation timed out
Skipping index 4584 due to translation error.


Translating dataset: 100%|██████████| 5000/5000 [2:30:10<00:00,  1.80s/it]

✅ All translations saved successfully to translated_dataset5k.txt


In [5]:
import re
import os

# File to clean
input_file = "translated_dataset5k.txt"

with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Function to clean text and fix filenames
def clean_line(line):
    # First, split between image name and caption
    parts = line.strip().split(" ", 1)
    
    if len(parts) != 2:
        return None  # Skip malformed lines

    image_name, caption = parts

    # Clean image name: handle formats like 'image___1...jpg'
    match = re.search(r'image_+(\d+)\.*jpg', image_name)
    if match:
        index = match.group(1)
        fixed_image_name = f"image_{index}.jpg"
    else:
        # Try to fallback to known pattern
        match = re.search(r'image(\d+)\.*jpg', image_name)
        if match:
            index = match.group(1)
            fixed_image_name = f"image_{index}.jpg"
        else:
            fixed_image_name = image_name  # keep as-is if unrecognized

    # Clean the caption: remove " and special characters (including Khmer period "។")
    caption = caption.replace('"', '')
    caption = re.sub(r"[!@#$%^&*()_+=\[\]{}\\|:;'<>,.?/~`។]", '', caption)
    caption = caption.strip()

    return f"{fixed_image_name} {caption}"

# Clean all lines
cleaned_lines = []
for line in lines:
    cleaned = clean_line(line)
    if cleaned:
        cleaned_lines.append(cleaned)

# Overwrite the original file
with open(input_file, "w", encoding="utf-8") as f:
    for line in cleaned_lines:
        f.write(line + "\n")

print(f"✅ Successfully cleaned and fixed filenames in {input_file}")


✅ Successfully cleaned and fixed filenames in translated_dataset5k.txt
